### 通过 `!pip list` 检查必要的python依赖库是否都存在

主要的两个python包是：

openai                          0.27.8

langchain                       0.0.200

In [3]:
!pip list

Package                           Version
--------------------------------- --------
actionlib                         1.14.0
aiohttp                           3.8.4
aiosignal                         1.3.1
angles                            1.9.13
anyio                             3.7.0
argilla                           1.11.0
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.3
asttokens                         2.2.1
async-lru                         2.0.2
async-timeout                     4.0.2
attrs                             23.1.0
Babel                             2.12.1
backcall                          0.2.0
backoff                           2.2.1
beautifulsoup4                    4.12.2
bleach                            6.0.0
bondpy                            1.8.6
camera-calibration                1.17.0
camera-calibration-parsers        1.12.0
catkin                            0.8.10
certifi                 

### 导入python依赖库、类和函数方法

In [2]:
import os
from langchain.llms import OpenAI

### 导入密钥（openai）

In [3]:
os.environ['OPENAI_API_KEY'] = 'sk-your-openai-api-key'

# <font color=red>项目1：构建最基本的文本对话</font>

 - 实例化一个大语言模型，这里实例化了 text-davinci-003 模型
 
 - 额外设置了参数 max_tokens=2048 ，其他参数按照类定义的默认值

In [6]:
BasicAskAnswer = OpenAI(model='text-davinci-003', max_tokens=2048)

 - 接下来开始编写prompt提示词。
 
> text-davinci-003 模型只能“一问一答”。如果需要能聊天的机器人，1）选用其他模型，例如 gpt-3.5-turbo；2）在prompt中增加上下文。

 - 最后输入到模型中等待回答。

In [7]:
prompt = """
            请你帮我设计一份中餐厅菜单。\n
            大概10种菜品，并设计出你的价格，以人民币为单位。\n
            请你以markdown语言格式化你的回答。
         """
response = BasicAskAnswer(prompt=prompt)
response

'\n1. 鱼香肉丝：38元\n2. 西红柿炒蛋：25元\n3. 蒜蓉粉丝蒸排骨：48元\n4. 宫保鸡丁：38元\n5. 麻婆豆腐：32元\n6. 糖醋排骨：45元\n7. 茄子炒肉片：35元\n8. 干锅鱼片：58元\n9. 甜酸茄子拌豆腐：25元\n10. 蔬菜拼盘：30元'

在提示词中我们要求大语言模型输出 markdown 格式，但是纯 print 不好看。

我们尝试使用 rich 库来辅助更漂亮的打印。

 - 先初始化一个虚拟控制台 csl ；
 - 讲原本是字符串的 response 转换成 markdown 实例
 - 将 markdown 实例输出到虚拟控制台 csl 。

In [8]:
import rich
from rich.console import Console
from rich.markdown import Markdown

In [9]:
csl = Console()
md = Markdown(response)
csl.print(md)

  1 鱼香肉丝：38元                                                                                                 
  2 西红柿炒蛋：25元                                                                                               
  3 蒜蓉粉丝蒸排骨：48元                                                                                           
  4 宫保鸡丁：38元                                                                                                 
  5 麻婆豆腐：32元                                                                                                 
  6 糖醋排骨：45元                                                                                                 
  7 茄子炒肉片：35元                                                                                               
  8 干锅鱼片：58元                                                                                                 
  9 甜酸茄子拌豆腐：25元                                                                                           
 10 蔬菜拼盘：30元

# <font color=red>项目2：长文本总结</font>

 - 学习外源文本转变成输入模型提示词的过程。
 
 - 外源文本 $\rightarrow$ 文档载入 $\rightarrow$ 转成document对象 $\rightarrow$ 文档切分 $\rightarrow$ 向量数据库。
 
 - 文本内容和这个jupyter notebook在同级目录，便可以简便导入。

In [10]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

In [12]:
# 导入文本
article_load = TextLoader("./content.txt")
# 转换成 document 对象
article_document = article_load.load()

这段代码创建了一个`RecursiveCharacterTextSplitter`对象，它是一个文本分割器，用于将长文本分割成较小的块。

`chunk_size=500`指定了每个块的大小为500个字符。

`chunk_overlap=0`指定了每个块之间没有重叠。

`Recursive`表示该分割器可以递归地将块继续分割成更小的块。

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)

接下来对文档进行切分

In [19]:
splitted_documents = text_splitter.split_documents(article_document)

加载大语言模型，直接拿上面实例化的 `BasicAskAnswer` 大语言模型复制过来，得到实例 `TextSummarizerChain`，这里还是默认使用刚才的 text-davinci-003 模型。

`refine`: 这种方式会先总结第一个 document，然后在将第一个 document 总结出的内容和第二个 document 一起发给 llm 模型在进行总结，以此类推。这种方式的好处就是在总结后一个 document 的时候，会带着前一个的 document 进行总结，给需要总结的 document 添加了上下文，增加了总结内容的连贯性。

`verbose`：为 `True` 时候打印出模型的思考信息。

In [20]:
TextSummarizer = BasicAskAnswer
TextSummarizerChain = load_summarize_chain(llm=TextSummarizer, chain_type='refine', verbose=True)

把分割好的全部文本送入实例 `TextSummarizerChain` 。

运行这个任务用了6分半，这可能是因为拆分文档的时候每一段分的太细了。

In [21]:
response = TextSummarizerChain.run(splitted_documents[:])
response



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


"All neighborhoods are not created equal when it comes to air quality.

Power plants, idling traffic, diesel trucks shipping goods and even the corner restaurant create different pollution signatures for different locations within cities and towns.

This summer, experts from Northeastern University are embarking on a multi-year quest to come up with block-by-block air quality insights in Boston-area municipalities.

The members of the interdisciplinary impact engine, known as iSUPER, will install more than 100 stationary pollution sensors in Brookline and Chelsea, and also explore Greater Boston streets in a van outfitted with monitoring equipment to detect greater variety of pollutants on a sub-neighborhood level. 

“Cities and towns have been making air pollution control decisions based on a few limited monitoring stations and regional averages,” says Yang Zhang,


> Finished chain.


> Entering new  chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: 
Northeastern University is launching a multi-year research project to collect block-by-block air quality data in the Greater Boston area in order to gain a better understanding of air quality and how it varies from neighborhood to neighborhood. The team is comprised of municipal officials and community groups, and has already installed 8 of the 100 solar-powered sensors in Brookline and Chelsea in collaboration with the Massachusetts Department of Environmental Protection (DEP). The remaining sensors will be placed through a collaborative process with municipal officials and community partners to identify hyperlocal hotspots of air pollution. The impact engine team currently consists of 13 faculty members in a variety of disciplines, one staff member, two research technicians, one co-op student, and 10 graduate studen


> Finished chain.

> Finished chain.


'\n\nNortheastern University is launching a multi-year research project, iSUPER (Intelligent Solutions to Urban Pollution for Equity and Resilience), to collect block-by-block air quality data in the Greater Boston area. The team is comprised of municipal officials, community groups, 13 faculty members from a variety of disciplines, one staff member, two research technicians, one co-op student, and 10 graduate students. It is set to expand with four new faculty members, three new staff members, several postdoctoral researchers, and 10 more graduate students in the coming year. The team has already installed 8 of the 100 solar-powered sensors in Brookline and Chelsea in collaboration with the Massachusetts Department of Environmental Protection (DEP), and plan to expand the project to additional locations. The project aims to establish a data portal where communities can access and explore air quality measurements from the stationary sensors and mobile lab, and to translate the raw data

同样地，用 rich 的 markdown 打印出来～

可以看到，用 text-davinci-003 和 load_summarize_chain 可以实现很长文档的总结！

In [22]:
md = Markdown(response)
csl.print(md)

Northeastern University is launching a multi-year research project, iSUPER (Intelligent Solutions to Urban         
Pollution for Equity and Resilience), to collect block-by-block air quality data in the Greater Boston area. The   
team is comprised of municipal officials, community groups, 13 faculty members from a variety of disciplines, one  
staff member, two research technicians, one co-op student, and 10 graduate students. It is set to expand with four 
new faculty members, three new staff members, several postdoctoral researchers, and 10 more graduate students in   
the coming year. The team has already installed 8 of the 100 solar-powered sensors in Brookline and Chelsea in     
collaboration with the Massachusetts Department of Environmental Protection (DEP), and plan to expand the project  
to additional locations. The project aims to establish a data portal where communities can access and explore air  
quality measurements from the stationary sensors and mobile lab, and to translate the raw data into insightful     
information. The van, wrapped in the university logo, will be equipped with research-grade air quality instruments 
that can measure ultrafine particles and other types of pollution that stationary sensors cannot detect. This      
partnership with Northeastern has the potential to save millions of lives around the globe, as high levels of air  
pollution can have severe health impacts on children, the elderly and low-income individuals. The information could
also help with city planning on multiple levels, such as understanding the impacts of adding green spaces, bike/bus
lanes, and very tall buildings. Even restaurant ventilation systems can affect local air quality, and recent       
studies have shown that air pollution levels could vary by five to eight times across a city block. The project    
aims to address these key gaps in understanding and build tools to help integrate air quality into all city design 
processes.

# <font color=red>项目3：基于Google的搜索问答</font>

- 进行Google搜索需要使用Google提供的API接口。

  教程如下：首先需要我们到 Serpapi 官网上注册一个用户，https://serpapi.com/ 并复制他给我们生成 api key。

- 一些主要库和对应版本列表如下：

  `google-search-results` ==2.4.2
  
- 安装一下必要的库

In [23]:
!pip install google-search-results==2.4.2

In [24]:
from langchain.agents import initialize_agent, load_tools, AgentType

导入能使用的 Google 搜索的 api 密钥

In [4]:
os.environ['SERPAPI_API_KEY'] = 'your-serpapi-api-key'

加载大语言模型，直接拿上面实例化的 `BasicAskAnswer` 大语言模型复制过来，得到实例 `SearchGoogle`，这里还是默认使用刚才的 text-davinci-003 模型。

In [26]:
SearchGoogle = BasicAskAnswer

因为基于Google的搜索回答很有可能是超出训练数据集外的。因此这里使用 langchain 的 agent ，让他根据设置好的 tool 来执行任务。

In [30]:
tools = load_tools(['serpapi'])

实例化智能体，把大语言模型、工具还有智能体类型都加进去做初始化。

In [31]:
agent = initialize_agent(llm=SearchGoogle, tools=tools, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

可见，基于Google的问答系统，可以获取到最新的国家的新闻。当然，你的prompt提示词要尽可能完善。

In [33]:
response = agent.run("什么是贸易顺差？")
response



> Entering new  chain...
 我需要搜索来获得一个准确的定义
Action: Search
Action Input: 贸易顺差
Observation: 所谓贸易顺差是指在特定年度一国出口贸易总额大于进口贸易总额，又称“出超”。 贸易顺差就是在一定的单位时间里（通常按年度计算），贸易的双方互相买卖各种货物，互相进口与出口，甲方的出口金额大过乙方的出口金额，或甲方的进口金额少于乙方的进口金额，其中的差额，对甲方来说，就叫作贸易顺差，反之，对乙方来说，就叫作贸易逆差。
Thought: 现在我知道答案了
Final Answer: 贸易顺差是指在特定年度一国出口贸易总额大于进口贸易总额，又称“出超”，其中的差额，对甲方来说，就叫作贸易顺差，反之，对乙方来说，就叫作贸易逆差。

> Finished chain.


'贸易顺差是指在特定年度一国出口贸易总额大于进口贸易总额，又称“出超”，其中的差额，对甲方来说，就叫作贸易顺差，反之，对乙方来说，就叫作贸易逆差。'

关于agent type 几个选项的含义：

`zero-shot-react-description`: 根据工具的描述和请求内容的来决定使用哪个工具（最常用）

`react-docstore`: 使用 ReAct 框架和 docstore 交互, 使用 `Search` 和 `Lookup` 工具, 前者用来搜, 后者寻找term, 举例: `Wipipedia` 工具

`self-ask-with-search`: 此智能体只使用一个工具: Intermediate Answer, 它会为问题寻找事实答案(指的非 gpt 生成的答案, 而是在网络中,文本中已存在的), 如 `Google search API` 工具

`conversational-react-description`: 为会话设置而设计的智能体, 它的prompt会被设计的具有会话性, 且还是会使用 ReAct 框架来决定使用来个工具, 并且将过往的会话交互存入内存

# <font color=red>项目4：基于本地知识库的问答机器人</font>

如何从我们本地读取多个文档构建知识库，并且使用 Openai API 在知识库中进行搜索并给出答案。

首先，导入一些必要依赖项。

In [34]:
!pip install unstructured
!pip install chromadb
!pip install tiktoken

这段代码是导入了 langchain 库中的一些模块和类，具体解释如下：

1. `from langchain.document_loaders import DirectoryLoader`：从 langchain 库中导入 `DirectoryLoader` 类，用于从文件夹中加载文档。
2. `from langchain.text_splitter import CharacterTextSplitter`：从 langchain 库中导入 `CharacterTextSplitter` 类，用于将文本分割成字符。
3. `from langchain.vectorstores import Chroma`：从 langchain 库中导入 `Chroma` 类，用于计算音乐的 Chroma 特征向量。
4. `from langchain.embeddings import OpenAIEmbeddings`：从 langchain 库中导入 `OpenAIEmbeddings` 类，用于获取 OpenAI 的预训练词向量。
5. `from langchain import OpenAI, VectorDBQA`：从 langchain 库中导入 `OpenAI` 和 `VectorDBQA` 类。
6. `from langchain.chains import RetrievalQA`：从 langchain 库中导入 `RetrievalQA` 类，用于实现检索式问答。

In [35]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQA

载入 `./Contents/` 文件中的 `.txt` 文件，并转换成 `document`

In [36]:
loader = DirectoryLoader(path='./Contents/', glob='**/*.txt')
document = loader.load()

初始化文本分割器，并分割文本。

In [37]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
documents = text_splitter.split_documents(document)

初始化 Open AI 的 Embedding 对象。

In [38]:
embedding = OpenAIEmbeddings()

通过 Open AI 的 Embedding 对象，将分割好的文本做成一个个向量数据，放入向量数据库。

In [39]:
docsearch = Chroma.from_documents(documents, embedding)

创建问答对象

In [41]:
from langchain.chains import RetrievalQA
QArobot = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type='stuff', vectorstore=docsearch, 
                                     return_source_documents=False,
                                     verbose=True)

In [45]:
# 进行问答
question = "本文的主人公是谁？他们在哪里？"
result = QArobot({"query": question})
result



> Entering new  chain...

> Finished chain.


{'query': '本文的主人公是谁？他们在哪里？',
 'result': ' The protagonists of this article are Fu Hongyuan, a PhD student from the class of 2022, and the members of the Major Railway Rock Burst Technology Research Group. They are at the construction site of a major railway project.'}

# <font color=red>项目5：基于持久知识库的问答机器人</font>

项目4的embedding在导入数据后需要计算一次；重启这个python就要反复计算，效率就降低了。

需要将本地文件转换成数据库，最好可以永久放在本地电脑上，这样重复加载时候就会快很多。

chroma 是个本地的向量数据库，他提供的一个 `persist_directory` 来设置持久化目录进行持久化。读取时，只需要调取 `from_document` 方法加载即可。

除此之外，还有：Pinecone 是一个在线的向量数据库。所以，我可以第一步依旧是注册，然后拿到对应的 api key。https://app.pinecone.io/ （不建议使用这个，需要付费）

大部分代码与项目4类似。在此处就做持久化的实现。

In [46]:
docsearch_persist = Chroma.from_documents(documents=documents, embedding=embedding, 
                                          persist_directory='./Contents/')  # 持久化标识
docsearch_persist.persist()
# 从持久库中加载数据
docsearch_persist = Chroma(persist_directory='./Contents/', embedding_function=embedding)

In [47]:
QArobot = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type='stuff', vectorstore=docsearch_persist, 
                                     return_source_documents=False,
                                     verbose=True)

In [48]:
# 进行问答
question = "本文中 He Benguo 做了什么？"
result = QArobot({"query": question})
result



> Entering new  chain...

> Finished chain.


{'query': '本文中 He Benguo 做了什么？',
 'result': ' He Benguo communicated with Fu Hongyuan in a timely manner, understood his living and research situation, and provided targeted guidance.'}

# <font color=red>项目6：使用GPT-3.5模型构建基于youtube网站的视频问答机器人</font>

设置了这个机器人，在浏览youtube的一些课程的时候，走神了就可以问这个机器人让他得到答案。

我猜基本原理是youtube有个自动生成字幕的功能，LangChain通过获取字幕内容来分析。

In [49]:
!pip install youtube-transcript-api

In [50]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import TextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ChatVectorDBChain, ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

加载 Youtube 里面的视频！

这里选用元强化学习 CBFinn 女士的一个 lecture 的视频～

https://www.youtube.com/watch?v=c0vSwglRY4w

(((Youtube 的广告是真的多。。。。。。

In [52]:
loader = YoutubeLoader.from_youtube_url(youtube_url='https://www.youtube.com/watch?v=c0vSwglRY4w')
youtube_document = loader.load()

In [53]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

In [54]:
youtube_documents = text_splitter.split_documents(youtube_document)

将 documents 放入向量数据库中

In [55]:
embedding = OpenAIEmbeddings()
vector_database = Chroma.from_documents(youtube_documents, embedding)
# 通过向量存储初始化检索器
retriever = vector_database.as_retriever()

因为是对话机器人，所以要初始化对话模板。

In [56]:
system_template = """
Use the following context to answer the user's question.
If you don't know the answer, say you don't, don't try to make it up. And answer in Chinese.
-----------
{chat_history}
"""

构建初始 messages 列表，这里可以理解为是 openai 传入的 messages 参数

In [57]:
messages = [SystemMessagePromptTemplate.from_template(system_template),
           HumanMessagePromptTemplate.from_template("{question}")]

初始化 prompt 对象

In [58]:
prompt = ChatPromptTemplate.from_messages(messages)

开始构建智能对话机器人

In [59]:
YoutubeQARobot = ConversationalRetrievalChain.from_llm(llm=ChatOpenAI(max_tokens=2048, temperature=0.5),
                                                       retriever=retriever,
                                                       condense_question_prompt=prompt)

开始进行对话～

In [61]:
chat_history = []
while True:
    question = input("问题：")
    if question == '':
        print("会话已经结束。")
        break
    result = YoutubeQARobot(
        {'question':question, 'chat_history':chat_history}
    )
    chat_history.append((question, result['answer']))
    print(result['answer'])

问题：你好！


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


你好！有什么我可以帮助你的吗？
问题：
会话已经结束。


# <font color=red>项目7：执行多个大语言链</font>

用于做更加多的工作～

In [62]:
from langchain.llms import OpenAI
from langchain.chains import SimpleSequentialChain, LLMChain
from langchain.prompts import PromptTemplate

In [63]:
llm_model = OpenAI(temperature=0.8)

这个是任务完成的第一步，也就是一个chain

In [67]:
# location 链
template = """
Your job is to come up with a classic dish from the area that the users suggests.

% USER LOCATION
{user_location}

YOUR RESPONSE:
"""

第一个chain的模板

In [68]:
first_prompt_template = PromptTemplate(input_variables=['user_location'], template=template)

构建第一个语言链任务

In [69]:
first_chain = LLMChain(llm=llm_model, prompt=first_prompt_template)

这个是任务完成的第二步，也就是第二个chain

In [70]:
# meal 链
template = """
Given a meal, give a short and simple recipe on how to make that dish at home.

% MEAL
{user_meal}

YOUR RESPONSE:
"""

In [73]:
second_prompt_template = PromptTemplate(input_variables=['user_meal'], template=template)

构建第二个chain

In [74]:
second_chain = LLMChain(llm=llm_model, prompt=second_prompt_template)

<font color=red>通过 SimpleSequentialChain 串联起来，第一个答案会被替换第二个中的user_meal，然后再进行询问</font>

In [80]:
overall_chain = SimpleSequentialChain(chains=[first_chain,second_chain],verbose=False)

In [82]:
result = overall_chain.run('Tailand')
csl.print(Markdown(result))

Pad Thai Recipe                                                                                                    

Ingredients:                                                                                                       

 • 4 ounces dried rice noodles                                                                                     
 • 2 tablespoons vegetable oil                                                                                     
 • 1/2 cup diced firm tofu                                                                                         
 • 2 large eggs, lightly beaten                                                                                    
 • 1/4 cup chopped peanuts                                                                                         
 • 1 cup bean sprouts                                                                                              
 • 2 tablespoons soy sauce                                                                                         
 • 2 tablespoons fish sauce                                                                                        
 • 2 tablespoons brown sugar                                                                                       
 • 2 tablespoons lime juice                                                                                        
 • 1/4 teaspoon red pepper flakes (optional)                                                                       

Instructions:                                                                                                      

 1 Bring a large pot of salted water to a boil over high heat. Add the noodles and cook until al dente, about 8    
   minutes. Drain and rinse with cold water.                                                                       
 2 Heat the oil in a wok or large skillet over high heat. Add the tofu and cook until lightly browned, about 5     
   minutes.                                                                                                        
 3 Push the tofu to the sides of the pan and pour the eggs into the center. Cook until the eggs are lightly        
   scrambled, about 2 minutes.                                                                                     
 4 Add the noodles, peanuts, bean sprouts, soy sauce, fish sauce, sugar, lime juice, and pepper flakes (if using)  
   to the pan. Stir everything together and cook until the noodles are hot, about 2 minutes.                       
 5 Serve

# <font color=red>项目8：结构化输出</font>

这样做的好处是，可以给后续做软件、做深度学习模型提供更加便捷的方案～～

In [83]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [84]:
llm_model = OpenAI(model='text-davinci-003',
                  max_tokens=2048,
                  temperature=0.5,
                  verbose=True)

In [85]:
# 告诉他我们生成的内容需要哪些字段，每个字段类型式啥
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

In [86]:
output_parsr = StructuredOutputParser.from_response_schemas(response_schemas)

In [91]:
format_instructions = output_parsr.get_format_instructions()
csl.print(Markdown(format_instructions))

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing 
"json" and "":                                                                                                     

{                                                                                                                  
    "bad_string": string  // This a poorly formatted user input string                                             
    "good_string": string  // This is your response, a reformatted response                                        
}                                                                                                                  

In [98]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

In [93]:
prompt = PromptTemplate(
    input_variables=['user_input'],
    partial_variables={'format_instructions':format_instructions},
    template=template
)

In [94]:
promptValue = prompt.format(user_input="welcom to califonya!")

In [96]:
llm_output = llm_model(promptValue)
csl.print(Markdown(llm_output))

{                                                                                                                  
    "bad_string": "welcom to califonya!",                                                                          
    "good_string": "Welcome to California!"                                                                        
}                                                                                                                  

使用刚才初始化好的解析器解析 llm_output 字符串。

In [99]:
total_result = output_parsr.parse(llm_output)
total_result

{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}

当然，我们也可以用rich库打印我们的结果。因为结果已经是一个字典了，所以打印字典就行。

In [100]:
from rich import print
print(total_result)

{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}

# <font color=red>项目9：爬取网页并生成json格式数据</font>

这样刷arxiv、IEEE上的论文就更快啦～～～

In [101]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMRequestsChain, LLMChain

In [102]:
llm_model = OpenAI(model='text-davinci-003',
                  max_tokens=1800,
                  temperature=1,
                  verbose=True)

In [103]:
template = """
在 >>> 和 <<< 之间是网页的返回的HTML内容。
网页是Arxiv网站中关于相关主题的论文简介。
请抽取参数请求的信息。

>>> {requests_result} <<<
请使用如下的JSON格式返回数据
{{
  "title":"a",
  "author":"b",
  "abstract":"c",
}}
Extracted:"""

In [104]:
prompt = PromptTemplate(input_variables=['requests_result'], template=template)

In [106]:
chain = LLMRequestsChain(
    llm_chain=LLMChain(llm=llm_model, prompt=prompt),
    verbose=False
)

In [107]:
inputs = {
  "url": "https://arxiv.org/search/?query=EEG&searchtype=all&source=header"
}

#### 使用小贴士

Regarding `openai.error.InvalidRequestError: This is a chat model and not supported in the v1/completions endpoint`

The code you posted above would work immediately if you change just one thing: `gpt-3.5-turbo` to `text-davinci-003`. 

This gives you an answer as to why you're getting this error. 

It's because you used the code that works with the `GPT-3` API endpoint, but wanted to use the `GPT-3.5` model (i.e., `gpt-3.5-turbo`). 

See model endpoint compatibility.

https://platform.openai.com/docs/models/how-we-use-your-data

In [108]:
response = chain(inputs)
print(response['output'])

"title": "Sparse wavelet-based solutions for the M/EEG inverse problem",
"author": "Samy Mokhtari, Jean-Michel Badier, Christian G. Bénar, Bruno Torrésani",
"abstract": "This paper is concerned with variational and Bayesian approaches to neuro-electromagnetic inverse 
problems (EEG and MEG). The strong indeterminacy of these problems is tackled by introducing sparsity inducing 
regularization/priors in a transformed domain, namely a spatial wavelet domain. Sparsity in the wavelet domain 
allows to reach ''data compression'' in the cortical sources domain. Spatial wavelets defined on the mesh graph of 
the triangulated cortical surface are used, in combination with sparse regression techniques, namely LASSO 
regression or sparse Bayesian learning, to provide localized and compressed estimates for brain activity from 
sensor data. Numerical results on simulated and real MEG data are provided, which outline the performances of the 
proposed approach in terms of localization."

我们可以优化一下，这样就能实现任意主题的爬取～～～

In [110]:
while True: 
    key_words_raw = input("请输入你的关键词：")
    if key_words_raw == '':
        print("没有输入关键词，已退出！")
        break
    key_words_raw = key_words_raw.split()
    start = '?query='
    end = '&searchtype=all&source=header'
    for i in range(len(key_words_raw)):
        if i != len(key_words_raw)-1:
            start = start + key_words_raw[i]+'+'
        else:
            start += key_words_raw[i]
    start += end
    address = "https://arxiv.org/search/"+start


    inputs = {
      "url": address
    }

    response = chain(inputs)
    print(response['output'])

请输入你的关键词：category level pose estimation


"title": "GenPose: Generative Category-level Object Pose Estimation via Diffusion Models",
"author": "Jiyao Zhang, Mingdong Wu, Hao Dong",
"abstract": "Object pose estimation plays a vital role in embodied AI and computer vision, enabling intelligent 
agents to comprehend and interact with their surroundings. Despite the practicality of category-level pose 
estimation, current approaches encounter challenges with partially observed point clouds, known as the 
multihypothesis issue. In this study, we propose a novel solution by reframing category-level object pose 
estimation as conditional generative modeling, departing from traditional point-to-point regression. Leveraging 
score-based diffusion models, we estimate object poses by sampling candidates from the diffusion model and 
aggregating them through a two-step process: filtering out outliers via likelihood estimation and subsequently 
mean-pooling the remaining candidates. To avoid the costly integration process when estimating the likelihood, we 
introduce an alternative method that trains an energy-based model from the original score-based model, enabling 
end-to-end likelihood estimation. Our approach achieves state-of-the-art performance on the REAL275 dataset, 
surpassing 50% and 60% on strict 5d2cm and 5d5cm metrics, respectively. Furthermore, our method demonstrates strong
generalizability to novel categories sharing similar symmetric properties without fine-tuning and can readily adapt
to object pose tracking tasks, yielding comparable results to the current state-of-the-art baselines."

请输入你的关键词：SLAM


{
  "title": "MOVESe: MOVablE and Moving LiDAR Scene Segmentation with Improved Navigation in Seg-label free 
settings",
  "author": "Prashant Kumar, Onkar Susladkar, Dhruv Makwana, Anurag Mittal, Prem Kumar Kalra",
  "abstract": "Accurate detection of movable and moving objects in LiDAR is of vital importance for navigation. 
Most existing works focus on extracting and removing moving objects during navigation. Movable objects like 
pedestrians, parked vehicles, etc. although static may move in the future. This leads to erroneous navigation and 
accidents. In such cases, it becomes necessary to detect potentially movable objects. To this end, we present a 
learning-based approach that segments movable and moving objects by generating static parts of scenes that are 
otherwise occluded. Our model performs superior to existing baselines on static LiDAR reconstructions using 3 
datasets including a challenging sparse industrial dataset. We achieve this without the assistance of any 
segmentation labels because such labels might not always be available for less popular yet important settings like 
industrial environments. The non-movable static parts of the scene generated by our model are of vital importance 
for downstream navigation for SLAM. The movable objects detected by our model can be fed to a downstream 3D 
detector for aiding navigation. Though we do not use segmentation, we evaluate our method against navigation 
baselines that use it to remove dynamic objects for SLAM. Through extensive experiments on several datasets, we 
showcase that our model surpasses these baselines on navigation.”
}

请输入你的关键词：segment


{
"title":"What a MESS: Multi-Domain Evaluation of Zero-Shot Semantic Segmentation",
"author":"Benedikt Blumenstiel, Johannes Jakubik, Hilde Kühne, Michael Vössing",
"abstract":"While semantic segmentation has seen tremendous improvements in the past, there is still significant 
labeling efforts necessary and the problem of limited generalization to classes that have not been present during 
training. To address this problem, zero-shot semantic segmentation makes use of large self-supervised 
vision-language models, allowing zero-shot transfer to unseen classes."
}

请输入你的关键词：


没有输入关键词，已退出！

# <font color=red>项目10：自定义agent中所使用的工具</font>

自定义工具里面有个比较有意思的地方，使用哪个工具的权重是**靠工具中描述内容**来实现的，和我们之前编程靠数值来控制权重完全不同。

In [111]:
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.llms import OpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, Tool, AgentType

In [112]:
llm = OpenAI(model='text-davinci-003',
            temperature=1,
            max_tokens=2048)

初始化搜索链和计算链

In [113]:
search = SerpAPIWrapper()
llm_math = LLMMathChain.from_llm(llm=llm, verbose=False)

创建一个功能列表，指明这个 agent 里面都有哪些可用工具

In [114]:
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math.run,
        description="useful for when you need to answer questions about math"
    )
]

初始化 agent

In [116]:
agent = initialize_agent(tools=tools,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, llm=llm)

最后就能执行 agent 啦～～

In [120]:
response = agent.run("周杰伦现在几岁？他的年龄乘以0.43是多少")
csl.print(Markdown(response))

Jay Chou is currently 39 years old and his age multiplied by 0.43 is 16.77.

# <font color=red>项目11：能一直聊天的对话机器人</font>

In [121]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

In [122]:
chat_model = ChatOpenAI(temperature=1, verbose=True)

因为聊天的本质实际是”聊天记录+当前的会话“，因此需要整合聊天历史。

In [123]:
chat_history = ChatMessageHistory()

In [124]:
# 给 MessageHistory 对象添加对话内容
chat_history.add_ai_message("你好！")
chat_history.add_user_message("中国的首都是哪里？")

In [125]:
# 执行对话
ai_response = chat_model(chat_history.messages)
ai_response.content

'中国的首都是北京。'

In [127]:
# 执行多轮对话
start_sentence = "你好！"
chat_history.add_ai_message(start_sentence)
print("机器人："+start_sentence)
while True: 
    user_message = input()
    if user_message == '':
        print("没有输入关键词，已退出！")
        break
    chat_history.add_user_message(user_message)
    ai_response = chat_model(chat_history.messages)
    ai_message = ai_response.content
    print("机器人："+ai_message)
    chat_history.add_ai_message(ai_message)

机器人：你好！

你好！


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_ret

KeyboardInterrupt: 